In [13]:
from dask.distributed import Client

client = Client(n_workers=4, threads_per_worker=2, memory_limit="4GB")
client

D:\ad_j3\venv\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53593 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:53593/status,
Dashboard: http://127.0.0.1:53593/status,Workers: 4
Total threads: 8,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53596,Workers: 4
Dashboard: http://127.0.0.1:53593/status,Total threads: 8
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:53615,Total threads: 2
Dashboard: http://127.0.0.1:53616/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:53599,


In [18]:
import dask_ml
import dask.array as da
from dask_ml.datasets import make_classification
from dask_ml.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from dask_ml.wrappers import Incremental


n, d = 10000000, 500
X, y = make_classification(n_samples=n, n_features=d,
                           chunks=n // 128, flip_y=0.2)
display(X)

X_train, X_test, y_train, y_test = train_test_split(X, y)
display(X_train)

classes = da.unique(y_train).compute()

est = SGDClassifier(loss='log_loss', penalty='l2', tol=1e-3)
inc = Incremental(est, scoring='accuracy')

inc.fit(X_train, y_train, classes=classes)
inc.score(X_test, y_test)

D:\ad_j3\venv\lib\site-packages\dask\base.py:1487: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


dask.array<normal, shape=(10000000, 500), dtype=float64, chunksize=(78125, 500), chunktype=numpy.ndarray>

dask.array<concatenate, shape=(8999936, 500), dtype=float64, chunksize=(70312, 500), chunktype=numpy.ndarray>

np.float64(0.5437052028670165)

In [19]:
import joblib
from sklearn.metrics import confusion_matrix

joblib.dump(inc, 'incremental_model.joblib')

inc_loaded = joblib.load('incremental_model.joblib')

y_pred = inc_loaded.predict(X_test)

cm = confusion_matrix(y_test.compute(), y_pred.compute())
print("Macierz klasyfikacji:")
print(cm)

Macierz klasyfikacji:
[[270500 228590]
 [227734 273240]]


In [20]:
from dask_ml.model_selection import IncrementalSearchCV
import numpy as np

X, y = make_classification(n_samples=5000000, n_features=20,
                            chunks=100000, random_state=56)

model = SGDClassifier(tol=1e-3, penalty='elasticnet', random_state=0)

params = {'alpha': np.logspace(-2, 1, num=1000),
          'l1_ratio': np.linspace(0, 1, num=1000),
          'average': [True, False]}

search = IncrementalSearchCV(model, params, random_state=0,
                             n_initial_parameters=1000,
                             patience=20,
                             max_iter=100)
search.fit(X, y, classes=[0, 1])
print("Najlepsze hiperparametry: ", search.best_params_)

D:\ad_j3\venv\lib\site-packages\dask\base.py:1487: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
D:\ad_j3\venv\lib\site-packages\dask_ml\model_selection\_incremental.py:1039: FutureWarning: decay_rate has been deprecated since Dask-ML v1.4.0.

    * Use InverseDecaySearchCV to use `decay_rate`
    * Specify decay_rate=None


  warn(


Najlepsze hiperparametry:  {'l1_ratio': np.float64(0.002002002002002002), 'average': False, 'alpha': np.float64(0.5711586478126432)}


In [29]:
print("Początkowe parametry modelu Zadanie 1:")
print(est.get_params())

Początkowe parametry modelu Zadanie 1:
{'alpha': 0.0001, 'average': False, 'class_weight': None, 'early_stopping': False, 'epsilon': 0.1, 'eta0': 0.0, 'fit_intercept': True, 'l1_ratio': 0.15, 'learning_rate': 'optimal', 'loss': 'log_loss', 'max_iter': 1000, 'n_iter_no_change': 5, 'n_jobs': None, 'penalty': 'l2', 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'tol': 0.001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
